# <center>**Taller 05:** Herramientas para indexación</center>

### Integrantes:
- Calahorrano David
- Córdova Carlos
- Zambrano Andrés

### 

### **Objetivo:**
Este ejercicio te permitirá recordar los conceptos fundamentales de indexación clásica en sistemas de recuperación de información. Implementarás un índice invertido manualmente y luego explorarás herramientas como Whoosh y Elasticsearch para construir y consultar índices.

### **Requisitos:**
1. Librerías Python necesarias: Whoosh, ElasticSearch.
2. Instalar Elasticsearch en un contenedor utilizando Docker.
3. Dataset Wikipedia Movie Plots, disponible en Kaggle.

-----

### Conceptos Clave
#### Indexación Invertida
- **Definición:** Un índice inverso es una estructura de datos que mapea cada palabra a la lista de documentos en los que aparece, junto con sus ubicaciones. Es fundamental en los sistemas de recuperación de información porque permite realizar búsquedas eficientes.
- **Ejemplo:** Si la palabra "cyborg" aparece en los documentos 1 y 3, el índice se vería así:
("cyborg", [1, 3])
- **Uso:** Facilita encontrar todos los documentos relevantes para una consulta.

#### Normalización de Texto
- **Definición:** Proceso de transformar texto para hacerlo uniforme, lo que incluye convertirlo a minúsculas, eliminar puntuación y caracteres especiales, entre otros.
- **Propósito:** Evitar inconsistencias en el procesamiento del texto (e.g., "Cyborg" y "cyborg" deben tratarse como iguales).
- **Pasos comunes:**
1. Convertir a minúsculas.
2. Eliminar puntuación.
3. Tokenizar en palabras.

#### Tokenización
- **Definición:** Dividir un texto en unidades básicas, generalmente palabras o frases.
- **Ejemplo:** El texto "Jurassic Park es una película." se tokeniza como:
["Jurassic", "park", "es", "una", "película"]
- **Herramientas:** En Python, librerías como NLTK o SpaCy pueden facilitar este proceso.

#### Whoosh
- **Definición:** Biblioteca de Python para construir motores de búsqueda ligeros, ideal para datasets pequeños.
- **Usos:** Indexación y búsqueda local sin necesidad de configurar servidores externos.
- **Ventajas:** Rápido y sencillo para proyectos individuales o educativos.

#### Elasticsearch
- **Definición:** Un motor de búsqueda de texto completo basado en Apache Lucene, diseñado para manejar grandes volúmenes de datos.

- **Usos:**
  - Crear índices distribuidos.
  - Realizar consultas complejas con operadores booleanos y puntuación de relevancia.

- **Ventajas:** Escalabilidad y velocidad en la recuperación de información en grandes colecciones de datos.

#### Consultas Básicas
- **Definición:** Proceso de buscar información en un índice utilizando palabras clave.
- **Ejemplos comunes:**
  - Match query: Buscar documentos que contengan una palabra clave.
  - Boolean query: Combinar palabras clave con operadores como "AND" u "OR".
- **Ejemplo con Elasticsearch:** Una consulta para buscar "dinosaurs AND park":
  ```json
  {
    "query": {
      "bool": {
        "must": [
          { "match": { "content": "dinosaurs" }},
          { "match": { "content": "park" }}
        ]
      }
    }
  }
  ```

---